# Scraping beer review site 

In [1]:
# Import dependencies for data wrangling  
import pandas as pd 
import numpy as np
import requests 
import json
from pprint import pprint

In [2]:
# Import dependencies for scraping
import pymongo
from time import sleep
from bs4 import BeautifulSoup

from splinter import Browser 
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
# Open the Chrome Driver Browser
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240


[WDM] - Driver [C:\Users\pooja\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


In [4]:
# Read in CSV
beer_df = pd.read_csv("../../Data/beer.gz", encoding="ISO-8859-1")
beer_df.head()

,Unnamed: 0,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,0,10325,Vecchio Birraio,2009-02-16 20:57:03,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,1,10325,Vecchio Birraio,2009-03-01 13:44:57,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,2,10325,Vecchio Birraio,2009-03-01 14:10:04,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,3,10325,Vecchio Birraio,2009-02-15 19:12:25,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,4,1075,Caldera Brewing Company,2010-12-30 18:53:26,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [5]:
# Drop the "unnamed: 0" column 
beer_df = beer_df.drop(['Unnamed: 0'], axis = 1)

In [6]:
beer_df.drop_duplicates(subset = "brewery_id", keep = "first", inplace = True)

In [7]:
beer_df = beer_df.loc[~(beer_df['brewery_id'] == 3444)]

In [8]:
beer_df = beer_df.reset_index(drop=True)

In [ ]:
beer_df

In [9]:
beer_df_2 = beer_df.iloc[438:5231]
beer_df_2.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
438,12591,Rogue's Roost Brew Pub & Eatery,2008-11-03 00:39:34,4.5,4.0,4.0,nighthawk,American Brown Ale,4.5,4.5,Bulldog Brown,5.00,37251
439,4304,Avesta Bryggeri,2005-09-29 18:48:08,3.5,4.0,4.0,bark,MÃ¤rzen / Oktoberfest,3.5,2.0,Avesta Rommehed Oktoberfestbier,5.00,25723
440,19455,Donnington Brewery,2009-02-28 19:05:02,2.5,2.0,2.5,wl0307,English Bitter,3.0,2.5,BB (Traditional Cotswold Bitter),3.60,48191
441,848,Bristol Brewing Company,2010-12-17 21:43:13,4.5,4.0,4.0,angrymice,Oatmeal Stout,3.5,5.0,Winter Warlock Oatmeal Stout,6.50,2883
442,11936,Sierra Madre Brewing Co.,2005-09-22 18:09:36,4.0,3.5,4.0,nekronos,KÃ¶lsch,3.5,4.0,Obispado,4.95,25925


In [10]:
brewery_ids_2 = beer_df_2["brewery_id"]
brewery_ids_2

438     12591
439      4304
440     19455
441       848
442     11936
        ...  
5224    21727
5225     6785
5226    12566
5227     7337
5228     6102
Name: brewery_id, Length: 4791, dtype: int64

In [ ]:
brewery_ids = beer_df["brewery_id"].unique()
type(brewery_ids)
len(brewery_ids)

In [ ]:
assert False

In [11]:
address_list = list()
count = 0

for brewery_id in brewery_ids_2:
    
        url = "https://www.beeradvocate.com/beer/profile/" + str(brewery_id) + "/"
        browser.visit(url)
        sleep(1)

        html = browser.html
        soup = BeautifulSoup(html, 'lxml')
        
        
        try:

            browser.links.find_by_partial_text('map').click()
            browser.windows[1].is_current = True
            sleep(1)

            html = browser.html 
            soup = BeautifulSoup(html, 'lxml')

            input_tag = soup.find(id = "searchboxinput")
            output = input_tag['value']

            address_list.append({
                "brewery_id": brewery_id,
                "address": output
            })

            count += 1
            print(count)
            
            browser.windows[1].close()

        except:
            pass 
        sleep(2)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=86.0.4240.111)


In [12]:
address_list

[{'brewery_id': 19455,
  'address': 'Donnington, Moreton-in-Marsh, GB2, UK, GL54 1EP'},
 {'brewery_id': 848,
  'address': '1604 S Cascade Ave, Colorado Springs, CO, US, 80906'},
 {'brewery_id': 11936,
  'address': 'Av. Vasconcelos 564 Ote., San Pedro, MX, 66250'},
 {'brewery_id': 17232, 'address': '440 County St., Milan, MI, US, 48160'},
 {'brewery_id': 9729, 'address': '83 Brighton Road, Glenelg, AU, 5045'},
 {'brewery_id': 6393,
  'address': 'Unit 8, Aizlewood Road,, Sheffield, GB2, UK, S8 0YX'},
 {'brewery_id': 1793, 'address': ', US'},
 {'brewery_id': 13866,
  'address': '1474 Kono, Tado-cho, Mie-ken Kuwana-shi, JP, 511-0122'},
 {'brewery_id': 24299, 'address': 'Ringstedvej 13, Svinninge, DK, 4520'},
 {'brewery_id': 27136, 'address': '46 Main Street, Bruthen, AU, 3885'},
 {'brewery_id': 1388,
  'address': 'Vinschgauerstr. 8 via Venosta, Merano, IT, 39022'},
 {'brewery_id': 12583, 'address': 'Taybeh, Ramallah, West Bank, PS'},
 {'brewery_id': 14955,
  'address': 'Rua Hermann Weege, 

In [13]:
address_df = pd.DataFrame(list(address_list), columns = ['brewery_id', 'address'])
address_df

,brewery_id,address
0,19455,"Donnington, Moreton-in-Marsh, GB2, UK, GL54 1EP"
1,848,"1604 S Cascade Ave, Colorado Springs, CO, US, ..."
2,11936,"Av. Vasconcelos 564 Ote., San Pedro, MX, 66250"
3,17232,"440 County St., Milan, MI, US, 48160"
4,9729,"83 Brighton Road, Glenelg, AU, 5045"
...,...,...
531,11437,"1 Cormorant Centre, Cormorant Drive, Runcorn, ..."
532,19247,"35 Hunua Rd, Auckland, NZ"
533,3062,"Common Road Industrial Estate, Stafford, GB2, ..."
534,870,"Laulupeo pst 15, Tartu, EE, 50050"


In [14]:
# Export the census data to csv to load to database
address_df.to_csv('second_set_address.csv', index=False)

In [ ]:
# Page URL that we are going to scrape
url = "https://www.beeradvocate.com/beer/profile/" + str(brewery_ids_2[438]) + "/"
print(url)
browser.visit(url) 

# Convention to let the script we are running sleep to allow everything to load 
sleep(0.5)

In [ ]:
# Interact with the map link
browser.links.find_by_partial_text('map').click()

In [ ]:
browser.windows[1].is_current = True
html = browser.html 
soup = BeautifulSoup(html, 'lxml')

In [ ]:
input_tag = soup.find(id = "searchboxinput")
output = input_tag['value']

In [ ]:
browser.windows[1].close()

In [ ]:
def listToString(s):
    
    address_string = ""
    
    for word in s:
        address_string += " " + word
    return address_string
 